In [1]:
%use dataframe(0.16.0-dev-6098)

This example uses the YouTube Data API: [https://developers.google.com/youtube/v3/docs](https://developers.google.com/youtube/v3/docs).
Follow the tutorials over there to gain an API key.

In [2]:
val apiKey = System.getenv("YOUTUBE_API_KEY")

In [3]:
fun load(path: String): AnyRow = DataRow.read("https://www.googleapis.com/youtube/v3/$path&key=$apiKey")

In [4]:
fun load(path: String, maxPages: Int): AnyFrame {
    val rows = mutableListOf<AnyRow>()
    var pagePath = path
    do {
        val row = load(pagePath)
        rows.add(row)
        val next = row.getValueOrNull<String>("nextPageToken")
        pagePath = path + "&pageToken=" + next
    } while (next != null && rows.size < maxPages)
    return rows.concat()
}

In [5]:
val df = load("search?q=cute%20cats&maxResults=50&part=snippet", 5)
df

<iframe onload="o_resize_iframe_out_1()" style="width:100%;" class="result_container" id="iframe_out_1" frameBorder="0" srcdoc=" <html>
 <head>
 <style type="text/css">
 :root {
 --background: #fff;
 --background-odd: #f5f5f5;
 --background-hover: #d9edfd;
 --header-text-color: #474747;
 --text-color: #848484;
 --text-color-dark: #000;
 --text-color-medium: #737373;
 --text-color-pale: #b3b3b3;
 --inner-border-color: #aaa;
 --bold-border-color: #000;
 --link-color: #296eaa;
 --link-color-pale: #296eaa;
 --link-hover: #1a466c;
}

:root[theme="dark"], :root [data-jp-theme-light="false"], .dataframe_dark{
 --background: #303030;
 --background-odd: #3c3c3c;
 --background-hover: #464646;
 --header-text-color: #dddddd;
 --text-color: #b3b3b3;
 --text-color-dark: #dddddd;
 --text-color-medium: #b2b2b2;
 --text-color-pale: #737373;
 --inner-border-color: #707070;
 --bold-border-color: #777777;
 --link-color: #008dc0;
 --link-color-pale: #97e1fb;
 --link-hover: #00688e;
}

p.dataframe_description {
 color: var(--text-color-dark);
}

table.dataframe {
 font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
 font-size: 12px;
 background-color: var(--background);
 color: var(--text-color-dark);
 border: none;
 border-collapse: collapse;
}

table.dataframe th, td {
 padding: 6px;
 border: 1px solid transparent;
 text-align: left;
}

table.dataframe th {
 background-color: var(--background);
 color: var(--header-text-color);
}

table.dataframe td {
 vertical-align: top;
 white-space: nowrap;
}

table.dataframe th.bottomBorder {
 border-bottom-color: var(--bold-border-color);
}

table.dataframe tbody > tr:nth-child(odd) {
 background: var(--background-odd);
}

table.dataframe tbody > tr:nth-child(even) {
 background: var(--background);
}

table.dataframe tbody > tr:hover {
 background: var(--background-hover);
}

table.dataframe a {
 cursor: pointer;
 color: var(--link-color);
 text-decoration: none;
}

table.dataframe tr:hover > td a {
 color: var(--link-color-pale);
}

table.dataframe a:hover {
 color: var(--link-hover);
 text-decoration: underline;
}

table.dataframe img {
 max-width: fit-content;
}

table.dataframe th.complex {
 background-color: var(--background);
 border: 1px solid var(--background);
}

table.dataframe .leftBorder {
 border-left-color: var(--inner-border-color);
}

table.dataframe .rightBorder {
 border-right-color: var(--inner-border-color);
}

table.dataframe .rightAlign {
 text-align: right;
}

table.dataframe .expanderSvg {
 width: 8px;
 height: 8px;
 margin-right: 3px;
}

table.dataframe .expander {
 display: flex;
 align-items: center;
}

/* formatting */

table.dataframe .null {
 color: var(--text-color-pale);
}

table.dataframe .structural {
 color: var(--text-color-medium);
 font-weight: bold;
}

table.dataframe .dataFrameCaption {
 font-weight: bold;
}

table.dataframe .numbers {
 color: var(--text-color-dark);
}

table.dataframe td:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}

table.dataframe tr:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}


:root {
 --scroll-bg: #f5f5f5;
 --scroll-fg: #b3b3b3;
}
:root[theme="dark"], :root [data-jp-theme-light="false"]{
 --scroll-bg: #3c3c3c;
 --scroll-fg: #97e1fb;
}
body {
 scrollbar-color: var(--scroll-fg) var(--scroll-bg);
}
body::-webkit-scrollbar {
 width: 10px; /* Mostly for vertical scrollbars */
 height: 10px; /* Mostly for horizontal scrollbars */
}
body::-webkit-scrollbar-thumb {
 background-color: var(--scroll-fg);
}
body::-webkit-scrollbar-track {
 background-color: var(--scroll-bg);
}
 </style>
 </head>
 <body>
 <table class="dataframe" id="df_-687865856"></table>

<p class="dataframe_description">DataFrame: rowsCount = 5, columnsCount = 7</p>

 </body>
 <script>
 (function () {
 window.DataFrame = window.DataFrame || new (function () {
 this.addTable = function (df) {
 let cols = df.cols;
 for (let i = 0; i < cols.length; i++) {
 for (let c of cols[i].children) {
 cols[c].parent = i;
 }
 }
 df.nrow = 0

In [6]:
val items = df.items.concat()
items

kind,etag,id,,snippet,,,,,,,,,,,,,,,
,,kind,videoId,publishedAt,channelId,title,description,thumbnails,,,,,,,,,channelTitle,liveBroadcastContent,publishTime
,,,,,,,,default,,,medium,,,high,,,,,
,,,,,,,,url,width,height,url,width,height,url,width,height,,,
youtube#searchResult,ieJNpaTNwWAVrRZTyXdyX_5taiw,youtube#video,DrpvkCRE2P4,2023-05-25T10:30:02Z,UCO5aEyYJeUXv8rxQEvbUMeA,Cute baby kitten sound ❤️ #shorts,,https://i.ytimg.com/vi/DrpvkCRE2P4/de...,120,90,https://i.ytimg.com/vi/DrpvkCRE2P4/mq...,320,180,https://i.ytimg.com/vi/DrpvkCRE2P4/hq...,480,360,Lovely Kittens,none,2023-05-25T10:30:02Z
youtube#searchResult,Ai7hOgkIKMs07ZGH8GsJDUNhS84,youtube#video,0KJS_2d2jOk,2024-08-23T23:46:27Z,UCXGm1fhynaYAFwaOxBSTPNg,13❤����Cute kitten #cat #lovecats #ca...,Cute kitten #cat #lovecats #cats #cat...,https://i.ytimg.com/vi/0KJS_2d2jOk/de...,120,90,https://i.ytimg.com/vi/0KJS_2d2jOk/mq...,320,180,https://i.ytimg.com/vi/0KJS_2d2jOk/hq...,480,360,Cat Kitten,none,2024-08-23T23:46:27Z
youtube#searchResult,XHnuMJew3mj2vm_OE5-vMOVEl_M,youtube#video,NsMKvVdEPkw,2023-05-06T14:18:05Z,UCDbVjtCeUnZliM4wTkhq2Lg,Kitten meowing to attract cats,"cat meowing to attract cats,cats meow...",https://i.ytimg.com/vi/NsMKvVdEPkw/de...,120,90,https://i.ytimg.com/vi/NsMKvVdEPkw/mq...,320,180,https://i.ytimg.com/vi/NsMKvVdEPkw/hq...,480,360,Cute Baby Cats 267,none,2023-05-06T14:18:05Z
youtube#searchResult,TkqVE28uNUVg47C4mRkfqER1ZM8,youtube#video,wdjpworLSk8,2023-09-15T22:02:00Z,UCQQcnrcfQbWvCl2lBvF5hoQ,Cute kittens❤️【30】#shorts #cutecat520...,shorts #catvideos #cat #funnycats #ki...,https://i.ytimg.com/vi/wdjpworLSk8/de...,120,90,https://i.ytimg.com/vi/wdjpworLSk8/mq...,320,180,https://i.ytimg.com/vi/wdjpworLSk8/hq...,480,360,Cute Cat��,none,2023-09-15T22:02:00Z
youtube#searchResult,Zbg-j8TEKj_VvmRu2CHZgmZRkuw,youtube#video,3bhkYoMWTFE,2023-07-26T13:33:50Z,UCkMrzpvOdM2Ndc21jgbj8NA,��Cats Doing Cat Things�� (3),cat #cats #catlover #catvideos #catlo...,https://i.ytimg.com/vi/3bhkYoMWTFE/de...,120,90,https://i.ytimg.com/vi/3bhkYoMWTFE/mq...,320,180,https://i.ytimg.com/vi/3bhkYoMWTFE/hq...,480,360,catvid-19,none,2023-07-26T13:33:50Z
youtube#searchResult,eRGtBGQW7bQjHH4WIr_I0NYD6wU,youtube#video,Po098TRdOn4,2023-01-02T09:46:11Z,UC6OZkZ7VcRGNMV1uQwnt9Gg,�� Funny cat videos | cute cats | Try...,Funny cat videos | cute cats | Try no...,https://i.ytimg.com/vi/Po098TRdOn4/de...,120,90,https://i.ytimg.com/vi/Po098TRdOn4/mq...,320,180,https://i.ytimg.com/vi/Po098TRdOn4/hq...,480,360,SRH Funny cats,none,2023-01-02T09:46:11Z
youtube#searchResult,-4j4KnkCwLxhwCAhh8X8PfDhoK4,youtube#video,y0sF5xhGreA,2023-05-28T13:00:44Z,UCPIvT-zcQl2H0vabdXJGcpg,20 Minutes of Adorable Kittens �� | B...,"Cute, cuddly, and utterly chaotic! Th...",https://i.ytimg.com/vi/y0sF5xhGreA/de...,120,90,https://i.ytimg.com/vi/y0sF5xhGreA/mq...,320,180,https://i.ytimg.com/vi/y0sF5xhGreA/hq...,480,360,The Pet Collective,none,2023-05-28T13:00:44Z


In [7]:
val videos = items.dropNulls { id.videoId }
    .select { id.videoId named "id" and snippet }
    .distinct()
videos

id,snippet,,,,,,,,,,,,,,,
,publishedAt,channelId,title,description,thumbnails,,,,,,,,,channelTitle,liveBroadcastContent,publishTime
,,,,,default,,,medium,,,high,,,,,
,,,,,url,width,height,url,width,height,url,width,height,,,
DrpvkCRE2P4,2023-05-25T10:30:02Z,UCO5aEyYJeUXv8rxQEvbUMeA,Cute baby kitten sound ❤️ #shorts,,https://i.ytimg.com/vi/DrpvkCRE2P4/de...,120,90,https://i.ytimg.com/vi/DrpvkCRE2P4/mq...,320,180,https://i.ytimg.com/vi/DrpvkCRE2P4/hq...,480,360,Lovely Kittens,none,2023-05-25T10:30:02Z
0KJS_2d2jOk,2024-08-23T23:46:27Z,UCXGm1fhynaYAFwaOxBSTPNg,13❤����Cute kitten #cat #lovecats #ca...,Cute kitten #cat #lovecats #cats #cat...,https://i.ytimg.com/vi/0KJS_2d2jOk/de...,120,90,https://i.ytimg.com/vi/0KJS_2d2jOk/mq...,320,180,https://i.ytimg.com/vi/0KJS_2d2jOk/hq...,480,360,Cat Kitten,none,2024-08-23T23:46:27Z
NsMKvVdEPkw,2023-05-06T14:18:05Z,UCDbVjtCeUnZliM4wTkhq2Lg,Kitten meowing to attract cats,"cat meowing to attract cats,cats meow...",https://i.ytimg.com/vi/NsMKvVdEPkw/de...,120,90,https://i.ytimg.com/vi/NsMKvVdEPkw/mq...,320,180,https://i.ytimg.com/vi/NsMKvVdEPkw/hq...,480,360,Cute Baby Cats 267,none,2023-05-06T14:18:05Z
wdjpworLSk8,2023-09-15T22:02:00Z,UCQQcnrcfQbWvCl2lBvF5hoQ,Cute kittens❤️【30】#shorts #cutecat520...,shorts #catvideos #cat #funnycats #ki...,https://i.ytimg.com/vi/wdjpworLSk8/de...,120,90,https://i.ytimg.com/vi/wdjpworLSk8/mq...,320,180,https://i.ytimg.com/vi/wdjpworLSk8/hq...,480,360,Cute Cat��,none,2023-09-15T22:02:00Z
3bhkYoMWTFE,2023-07-26T13:33:50Z,UCkMrzpvOdM2Ndc21jgbj8NA,��Cats Doing Cat Things�� (3),cat #cats #catlover #catvideos #catlo...,https://i.ytimg.com/vi/3bhkYoMWTFE/de...,120,90,https://i.ytimg.com/vi/3bhkYoMWTFE/mq...,320,180,https://i.ytimg.com/vi/3bhkYoMWTFE/hq...,480,360,catvid-19,none,2023-07-26T13:33:50Z
Po098TRdOn4,2023-01-02T09:46:11Z,UC6OZkZ7VcRGNMV1uQwnt9Gg,�� Funny cat videos | cute cats | Try...,Funny cat videos | cute cats | Try no...,https://i.ytimg.com/vi/Po098TRdOn4/de...,120,90,https://i.ytimg.com/vi/Po098TRdOn4/mq...,320,180,https://i.ytimg.com/vi/Po098TRdOn4/hq...,480,360,SRH Funny cats,none,2023-01-02T09:46:11Z
y0sF5xhGreA,2023-05-28T13:00:44Z,UCPIvT-zcQl2H0vabdXJGcpg,20 Minutes of Adorable Kittens �� | B...,"Cute, cuddly, and utterly chaotic! Th...",https://i.ytimg.com/vi/y0sF5xhGreA/de...,120,90,https://i.ytimg.com/vi/y0sF5xhGreA/mq...,320,180,https://i.ytimg.com/vi/y0sF5xhGreA/hq...,480,360,The Pet Collective,none,2023-05-28T13:00:44Z


In [8]:
val parsed = videos.parse()

In [9]:
val loaded = parsed.convert { colsAtAnyDepth().colsOf<URL>() }.with { IMG(it, maxHeight = 150) }
    .add("video") { IFRAME("https://www.youtube.com/embed/$id") }

NOTE: For this example, the DataFrame needs to be rendered as HTML. This means that when running in Kotlin Notebook, "Render DataFrame tables natively" needs to be turned off.

In [10]:
val clean = loaded.move { snippet.channelId and snippet.channelTitle }.under("channel")
    .move { snippet.title and snippet.publishedAt }.toTop()
    .remove { snippet }
clean

id,title,publishedAt,channel,,video
,,,channelId,channelTitle,
DrpvkCRE2P4,Cute baby kitten sound ❤️ #shorts,2023-05-25T10:30:02Z,UCO5aEyYJeUXv8rxQEvbUMeA,Lovely Kittens,"<iframe src=""https://www.youtube.com/..."
0KJS_2d2jOk,13❤����Cute kitten #cat #lovecats #ca...,2024-08-23T23:46:27Z,UCXGm1fhynaYAFwaOxBSTPNg,Cat Kitten,"<iframe src=""https://www.youtube.com/..."
NsMKvVdEPkw,Kitten meowing to attract cats,2023-05-06T14:18:05Z,UCDbVjtCeUnZliM4wTkhq2Lg,Cute Baby Cats 267,"<iframe src=""https://www.youtube.com/..."
wdjpworLSk8,Cute kittens❤️【30】#shorts #cutecat520...,2023-09-15T22:02:00Z,UCQQcnrcfQbWvCl2lBvF5hoQ,Cute Cat��,"<iframe src=""https://www.youtube.com/..."
3bhkYoMWTFE,��Cats Doing Cat Things�� (3),2023-07-26T13:33:50Z,UCkMrzpvOdM2Ndc21jgbj8NA,catvid-19,"<iframe src=""https://www.youtube.com/..."
Po098TRdOn4,�� Funny cat videos | cute cats | Try...,2023-01-02T09:46:11Z,UC6OZkZ7VcRGNMV1uQwnt9Gg,SRH Funny cats,"<iframe src=""https://www.youtube.com/..."
y0sF5xhGreA,20 Minutes of Adorable Kittens �� | B...,2023-05-28T13:00:44Z,UCPIvT-zcQl2H0vabdXJGcpg,The Pet Collective,"<iframe src=""https://www.youtube.com/..."
NlxYY7w1UGU,Cat videos cute cats kittens ����,2024-01-08T13:13:21Z,UCTmml_z2rBmxDUA9Ie4o_uA,Little Kitten 21,"<iframe src=""https://www.youtube.com/..."
a7K-kWT_C2A,Funny Cute Cats ��,2023-07-22T05:27:58Z,UCsVD3ZguqePDEKaPHUeaIuQ,ThomasTrucker,"<iframe src=""https://www.youtube.com/..."


In [11]:
val statPages = clean.id.chunked(50).map {
    val ids = it.joinToString("%2C")
    load("videos?part=statistics&id=$ids")
}
statPages

<iframe onload="o_resize_iframe_out_8()" style="width:100%;" class="result_container" id="iframe_out_8" frameBorder="0" srcdoc=" <html>
 <head>
 <style type="text/css">
 :root {
 --background: #fff;
 --background-odd: #f5f5f5;
 --background-hover: #d9edfd;
 --header-text-color: #474747;
 --text-color: #848484;
 --text-color-dark: #000;
 --text-color-medium: #737373;
 --text-color-pale: #b3b3b3;
 --inner-border-color: #aaa;
 --bold-border-color: #000;
 --link-color: #296eaa;
 --link-color-pale: #296eaa;
 --link-hover: #1a466c;
}

:root[theme="dark"], :root [data-jp-theme-light="false"], .dataframe_dark{
 --background: #303030;
 --background-odd: #3c3c3c;
 --background-hover: #464646;
 --header-text-color: #dddddd;
 --text-color: #b3b3b3;
 --text-color-dark: #dddddd;
 --text-color-medium: #b2b2b2;
 --text-color-pale: #737373;
 --inner-border-color: #707070;
 --bold-border-color: #777777;
 --link-color: #008dc0;
 --link-color-pale: #97e1fb;
 --link-hover: #00688e;
}

p.dataframe_description {
 color: var(--text-color-dark);
}

table.dataframe {
 font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
 font-size: 12px;
 background-color: var(--background);
 color: var(--text-color-dark);
 border: none;
 border-collapse: collapse;
}

table.dataframe th, td {
 padding: 6px;
 border: 1px solid transparent;
 text-align: left;
}

table.dataframe th {
 background-color: var(--background);
 color: var(--header-text-color);
}

table.dataframe td {
 vertical-align: top;
 white-space: nowrap;
}

table.dataframe th.bottomBorder {
 border-bottom-color: var(--bold-border-color);
}

table.dataframe tbody > tr:nth-child(odd) {
 background: var(--background-odd);
}

table.dataframe tbody > tr:nth-child(even) {
 background: var(--background);
}

table.dataframe tbody > tr:hover {
 background: var(--background-hover);
}

table.dataframe a {
 cursor: pointer;
 color: var(--link-color);
 text-decoration: none;
}

table.dataframe tr:hover > td a {
 color: var(--link-color-pale);
}

table.dataframe a:hover {
 color: var(--link-hover);
 text-decoration: underline;
}

table.dataframe img {
 max-width: fit-content;
}

table.dataframe th.complex {
 background-color: var(--background);
 border: 1px solid var(--background);
}

table.dataframe .leftBorder {
 border-left-color: var(--inner-border-color);
}

table.dataframe .rightBorder {
 border-right-color: var(--inner-border-color);
}

table.dataframe .rightAlign {
 text-align: right;
}

table.dataframe .expanderSvg {
 width: 8px;
 height: 8px;
 margin-right: 3px;
}

table.dataframe .expander {
 display: flex;
 align-items: center;
}

/* formatting */

table.dataframe .null {
 color: var(--text-color-pale);
}

table.dataframe .structural {
 color: var(--text-color-medium);
 font-weight: bold;
}

table.dataframe .dataFrameCaption {
 font-weight: bold;
}

table.dataframe .numbers {
 color: var(--text-color-dark);
}

table.dataframe td:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}

table.dataframe tr:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}


:root {
 --scroll-bg: #f5f5f5;
 --scroll-fg: #b3b3b3;
}
:root[theme="dark"], :root [data-jp-theme-light="false"]{
 --scroll-bg: #3c3c3c;
 --scroll-fg: #97e1fb;
}
body {
 scrollbar-color: var(--scroll-fg) var(--scroll-bg);
}
body::-webkit-scrollbar {
 width: 10px; /* Mostly for vertical scrollbars */
 height: 10px; /* Mostly for horizontal scrollbars */
}
body::-webkit-scrollbar-thumb {
 background-color: var(--scroll-fg);
}
body::-webkit-scrollbar-track {
 background-color: var(--scroll-bg);
}
 </style>
 </head>
 <body>
 <table class="dataframe" id="df_-687865822"></table>

<p class="dataframe_description">DataFrame: rowsCount = 5, columnsCount = 4</p>

 </body>
 <script>
 (function () {
 window.DataFrame = window.DataFrame || new (function () {
 this.addTable = function (df) {
 let cols = df.cols;
 for (let i = 0; i < cols.length; i++) {
 for (let c of cols[i].children) {
 cols[c].parent = i;
 }
 }
 df.nrow = 0

In [12]:
val stats = statPages.items.concat().select { id and statistics.allCols() }.parse()
stats

id,viewCount,likeCount,favoriteCount,commentCount
DrpvkCRE2P4,53942062,1095095,0,5286
0KJS_2d2jOk,54431535,874236,0,3828
NsMKvVdEPkw,264107598,3256456,0,11995
wdjpworLSk8,118559669,1900376,0,6928
3bhkYoMWTFE,400315589,9621364,0,59131
Po098TRdOn4,50216211,864405,0,3523
y0sF5xhGreA,16896292,61992,0,2363
NlxYY7w1UGU,70900051,1807692,0,10668
a7K-kWT_C2A,84705746,3144429,0,18183
ASFCURj-2hQ,44898078,null,0,1342


In [13]:
val joined = clean.join(stats)
joined

id,title,publishedAt,channel,,video,viewCount,likeCount,favoriteCount,commentCount
,,,channelId,channelTitle,,,,,
DrpvkCRE2P4,Cute baby kitten sound ❤️ #shorts,2023-05-25T10:30:02Z,UCO5aEyYJeUXv8rxQEvbUMeA,Lovely Kittens,"<iframe src=""https://www.youtube.com/...",53942062,1095095,0,5286
0KJS_2d2jOk,13❤����Cute kitten #cat #lovecats #ca...,2024-08-23T23:46:27Z,UCXGm1fhynaYAFwaOxBSTPNg,Cat Kitten,"<iframe src=""https://www.youtube.com/...",54431535,874236,0,3828
NsMKvVdEPkw,Kitten meowing to attract cats,2023-05-06T14:18:05Z,UCDbVjtCeUnZliM4wTkhq2Lg,Cute Baby Cats 267,"<iframe src=""https://www.youtube.com/...",264107598,3256456,0,11995
wdjpworLSk8,Cute kittens❤️【30】#shorts #cutecat520...,2023-09-15T22:02:00Z,UCQQcnrcfQbWvCl2lBvF5hoQ,Cute Cat��,"<iframe src=""https://www.youtube.com/...",118559669,1900376,0,6928
3bhkYoMWTFE,��Cats Doing Cat Things�� (3),2023-07-26T13:33:50Z,UCkMrzpvOdM2Ndc21jgbj8NA,catvid-19,"<iframe src=""https://www.youtube.com/...",400315589,9621364,0,59131
Po098TRdOn4,�� Funny cat videos | cute cats | Try...,2023-01-02T09:46:11Z,UC6OZkZ7VcRGNMV1uQwnt9Gg,SRH Funny cats,"<iframe src=""https://www.youtube.com/...",50216211,864405,0,3523
y0sF5xhGreA,20 Minutes of Adorable Kittens �� | B...,2023-05-28T13:00:44Z,UCPIvT-zcQl2H0vabdXJGcpg,The Pet Collective,"<iframe src=""https://www.youtube.com/...",16896292,61992,0,2363
NlxYY7w1UGU,Cat videos cute cats kittens ����,2024-01-08T13:13:21Z,UCTmml_z2rBmxDUA9Ie4o_uA,Little Kitten 21,"<iframe src=""https://www.youtube.com/...",70900051,1807692,0,10668
a7K-kWT_C2A,Funny Cute Cats ��,2023-07-22T05:27:58Z,UCsVD3ZguqePDEKaPHUeaIuQ,ThomasTrucker,"<iframe src=""https://www.youtube.com/...",84705746,3144429,0,18183


In [14]:
val view by column<Int>()

val channels = joined.groupBy { channel }.sortByCount().aggregate {
    viewCount.sum() into view

    val last = maxBy { publishedAt }
    last.title into "last title"
    last.publishedAt into "time"
    last.viewCount into "viewCount"
}.sortByDesc(view).flatten()
channels

channelId,channelTitle,view,last title,time,viewCount
UCS56r87Y7q1SrAB-42brE-w,Sonyakisa8 TT,1615887472,Fears of Sonya&#39;s cat��#cat #cats,2023-09-18T11:56:57Z,225916305
UCvlE5gTbOvjiolFlEm-c_Ow,Vlad and Niki,438978403,Vlad and Niki play with Kittens,2020-05-16T07:17:55Z,438978403
UCzybj8GhOrPvWv_ZctJAlAw,Prince Tom,429657006,I lost my kitten on the street�� #cat...,2024-07-17T02:00:12Z,429657006
UCkMrzpvOdM2Ndc21jgbj8NA,catvid-19,400315589,��Cats Doing Cat Things�� (3),2023-07-26T13:33:50Z,400315589
UCuSkK03h80AhO4IkvaSWdow,exlittlebeans,278587037,"Boom,Scared me! #exlittlebeans #funny...",2023-08-12T14:26:13Z,225174626
UCrj6b6NZy_EY_hmIfI9e63Q,Catfuntv,267018897,"revenge of dad ���� , Cat �� lion , #...",2024-04-01T10:31:46Z,20823101
UCDbVjtCeUnZliM4wTkhq2Lg,Cute Baby Cats 267,264107598,Kitten meowing to attract cats,2023-05-06T14:18:05Z,264107598
UC0HRkOUeR4l1qwpRf9gLmdA,Lovie Jennaira,236184147,Cute cats and kittens always take my ...,2024-11-17T02:36:09Z,721089
UCI3-YbYcx-rfnQjwU9P9fwg,Lovento Films,219054509,Spooky Clown is after cute Cat and Ki...,2024-11-18T08:48:39Z,36137996
UCV-UkiKP93gACw7OX-0pJ1g,Pet Lovers,210070700,Little mao #cat #catvideos #shorts,2021-11-05T03:00:02Z,210070700


In [15]:
%useLatestDescriptors
%use kandy

In [16]:
channels.sortBy { viewCount.desc() }.plot {
    bars {
        x(channelTitle.map { it.take(10) })
        y(viewCount)
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="008qeL"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"mapping":{
},
"data":{
"viewCount":[4.38978403E8,4.29657006E8,4.00315589E8,2.64107598E8,2.25916305E8,2.25174626E8,2.100707E8,1.69567181E8,1.18559669E8,9.1415057E7,8.6045613E7,8.4705746E7,7.8234649E7,7.3427203E7,6.5644554E7,5.7748332E7,5.4431535E7,5.3607665E7,5.0216211E7,4.4898078E7,4.466718E7,4.448825E7,4.3165962E7,3.9167015E7,3.8527138E7,3.7046599E7,3.6906565E7,3.6137996E7,3.4180106E7,3.2285554E7,3.1548929E7,2.9832773E7,2.9343981E7,2.679383E7,2.6676049E7,2.5362311E7,2.369168E7,2.3545656E7,2.2895934E7,2.2744819E7,2.1313232E7,2.0823101E7,2.0813044E7,1.8850263E7,1.8630496E7,1.8436952E7,1.7586725E7,1.5702508E7,1.4955544E7,1.4878381E7,1.4811646E7,1.472365E7,1.3969405E7,1.3635539E7,1.3413824E7,1.339115E7,1.3132056E7,1.2597336E7,1.1219877E7,1.0698645E7,1.0166747E7,9505994.0,9188961.0,8989412.0,8241923.0,7617758.0,7538872.0,7374973.0,7361395.0,7220635.0,6533897.0,6258329.0,6136139.0,5862079.0,5173746.0,5099900.0,4749708.0,4553605.0,4346694.0,4274140.0,3933157.0,3851211.0,3694977.0,3420484.0,3390568.0,2978717.0,2679703.0,2354353.0,2304885.0,2211633.0,1883049.0,1865690.0,1807179.0,1726303.0,1501405.0,1430632.0,1288366.0,1254913.0,1096238.0,1072193.0,1039715.0,1036206.0,995731.0,742125.0,726420.0,721089.0,660161.0,637626.0,622241.0,598391.0,569477.0,505175.0,476183.0,436475.0,361981.0,301173.0,296677.0,167976.0,118745.0,118208.0,76123.0,70525.0,68289.0,64304.0,57108.0,24334.0,18936.0,15383.0,15184.0,11231.0,10576.0,10104.0,6229.0,5307.0,4711.0,4183.0,2933.0,2179.0,283.0],
"channelTitle":["Vlad and N","Prince Tom","catvid-19","Cute Baby ","Sonyakisa8","exlittlebe","Pet Lovers","Baby Boom-"," Cute Cat?","VIKU INFO","Bunch Of V","ThomasTruc","susan chan","Ana.simon_","Oscar's Fu","NoCAT NoLi","Cat Kitten","Stocat","SRH Funny ","Toon4","The geeta ","Cartoon tu","dragon hub","Cats being","Princess c","Aww Animal","Funny-anim","Lovento Fi","Cat 😍 sho","Baby Cat F","Prank","SydVids","FunnyCatTv","Prince Lou","Republic O","Maiw Milky","FUN Bong","Rainbow Ju","CUTE CAT","The BN Fam","ArtTime Id","Catfuntv","DIARY DIMU","Shefali Ra","Lulu the C","Jeff Canal","Diary Pets","PurrfectPa","Rufus","Eevee & Yo","Mantio","Funny Rhyt","Marble Run","Lovely Kit","Daily Dose","Ai Game ","Brennan Ro","Aymie and ","Cat Cute J","Si Lavi","SugarStar","CatAniki","Funniest A","土豆の日記Cat's","The Pet Zo","Furry Corn","Pretty Kit","Happy Pets","Cute Cat o","FurryFritz","COSEFNASTY","Organic Na","meowneow","Teddy Kitt","QT Katz","MIAW MILKY","Pets Momen","BN ki Mast","Ho salt","Ginger Cat","Incredible","Kitty Show","Pets Funny","The Meow S","Kitten Won","Baby viral","pecintatom","~Miss Vent","Kak Nurul","Meow Cat S","Cute Wild ","Ninja Cats","CUTE ANIMA","CatClips","Top 10 Ser","Cute Anima","Cats Milky","The Pet Co","Koray Zeyn","Cat Fun 0","Cute Cat o","21","Daily Dose","BearyDeliv","Arkhan Ras","Lovie Jenn","Funny Funn","StickyToon","Dancing Ca","CuteAnimal","Funny And ","Cute Cat o","Purrallel ","RB adorabl","CATS on GP","@Mini Shor","Little Lov","Little Kit","7 Cat Ai","Cute cat's","Teddy Feli","ميجي - MIG","DonPaulieW","HelloMochi","Funny Toon","Meow Mirth","CatFun","BitWorldTV","Cat story ","Cute cats","Melisukaku","Masser Meo","Pawspace","Ai meomeo","SynthPhori","DST_deepak","Hendri 088","Funny Arra","Fun zone o"]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"channelTitle",
"y":"viewCount"
},
"stat":"identity",
"sampling":"none",
"inherit_aes":false,
"position":"dodge",
"geom":"bar",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"str",
"column":"cha

In [17]:
val growth = joined
    .select { publishedAt and viewCount }
    .sortBy { publishedAt }
    .convert { all() }.toLong()
    .cumSum { viewCount }

In [18]:
growth.plot {
    area {
        x(publishedAt)
        y(viewCount)
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="Kzoo3s"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"mapping":{
},
"data":{
"publishedAt":[1.441432817E12,1.465995604E12,1.577569041E12,1.580917785E12,1.586534428E12,1.589613475E12,1.62193824E12,1.624121248E12,1.630931238E12,1.634841773E12,1.636081202E12,1.639155613E12,1.645155639E12,1.649927372E12,1.67239445E12,1.672652771E12,1.674361058E12,1.674756662E12,1.675587023E12,1.677005928E12,1.677071477E12,1.67897218E12,1.679617812E12,1.680254122E12,1.680448798E12,1.682629389E12,1.68301981E12,1.683382685E12,1.684461611E12,1.685010602E12,1.685278844E12,1.685535879E12,1.685795773E12,1.686549969E12,1.687430453E12,1.688909778E12,1.68956107E12,1.690003678E12,1.69037843E12,1.691544601E12,1.691850373E12,1.692276804E12,1.693747819E12,1.69481532E12,1.695038217E12,1.69641004E12,1.696795206E12,1.698405303E12,1.70042041E12,1.701072256E12,1.703490944E12,1.704719601E12,1.70653662E12,1.708473604E12,1.708490683E12,1.70895241E12,1.709903125E12,1.710417621E12,1.710758999E12,1.710767884E12,1.710860433E12,1.711622705E12,1.711710029E12,1.711713639E12,1.711967506E12,1.712127586E12,1.712294024E12,1.713379367E12,1.714315225E12,1.715415313E12,1.716129479E12,1.716518623E12,1.718425959E12,1.7188905E12,1.718968589E12,1.719423661E12,1.719491004E12,1.7200821E12,1.721181612E12,1.721420353E12,1.721650763E12,1.721708861E12,1.7219085E12,1.72212861E12,1.722434406E12,1.722733213E12,1.722759563E12,1.723221904E12,1.723259001E12,1.723538203E12,1.72372879E12,1.723740856E12,1.723874413E12,1.724119762E12,1.724373308E12,1.724456787E12,1.724553019E12,1.7246391E12,1.725094828E12,1.725198562E12,1.725453011E12,1.725483945E12,1.7255433E12,1.725614511E12,1.725627618E12,1.725795005E12,1.726012835E12,1.726107304E12,1.726138845E12,1.726336816E12,1.7264523E12,1.726617618E12,1.72662271E12,1.726717616E12,1.72688222E12,1.726882668E12,1.7271837E12,1.727316016E12,1.727322677E12,1.727362825E12,1.727430478E12,1.727654415E12,1.728807538E12,1.72921518E12,1.729256979E12,1.72939362E12,1.729900823E12,1.73004093E12,1.730082902E12,1.730321276E12,1.730509204E12,1.730683829E12,1.730778609E12,1.730876206E12,1.730925138E12,1.731039182E12,1.731810969E12,1.731852031E12,1.731919719E12,1.732152857E12,1.732668879E12,1.732705221E12,1.732773608E12,1.732856404E12,1.733026476E12,1.733757369E12,1.733825741E12,1.733842592E12,1.733918603E12,1.733925619E12,1.73437691E12,1.734616836E12,1.734637943E12,1.734693304E12,1.734739205E12,1.734741189E12,1.73478306E12,1.73491557E12,1.735012832E12,1.735051582E12,1.735236523E12,1.735357969E12,1.735381818E12,1.735390831E12,1.735736438E12,1.735996877E12,1.736071994E12,1.736172024E12,1.736330435E12,1.736349224E12,1.736828497E12,1.736828662E12,1.736998202E12,1.737000963E12,1.737007044E12,1.737049131E12,1.737115318E12,1.737196291E12,1.737252047E12,1.737314107E12,1.737423717E12,1.737457283E12,1.737606174E12,1.737629134E12,1.73769485E12,1.73775942E12,1.737818101E12,1.73795041E12,1.737973764E12,1.738298418E12,1.738303297E12,1.738504846E12,1.738816021E12,1.74063433E12,1.740661479E12,1.740726011E12,1.740832032E12,1.740873629E12,1.7409243E12,1.740956405E12,1.74101442E12,1.741061705E12,1.7410689E12,1.741071608E12,1.741097978E12,1.741154402E12,1.741177456E12,1.741179231E12,1.741184881E12,1.741185048E12,1.741213004E12,1.741258835E12,1.741281066E12,1.741339085E12],
"viewCount":[5.7748332E7,7.11787E7,7.7040779E7,1.14087378E8,1.29042922E8,5.68021325E8,5.76263248E8,5.78070427E8,5.92882073E8,6.05479409E8,8.15550109E8,8.39095765E8,8.7826278E8,8.8253692E8,1.052104101E9,1.102320312E9,1.193735369E9,1.20861375E9,1.210496799E9,1.233213022E9,1.292866721E9,1.530251027E9,1.536036636E9,1.572943201E9,1.626355612E9,1.633576247E9,1.638945154E9,1.903052